In [76]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [77]:
# open text file and read in data as `text`
with open('anna.txt', 'r') as f:
    text = f.read()

In [78]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [79]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])

In [80]:
encoded[:100]

array([50, 36, 57, 25, 14, 64, 35, 19, 34, 42, 42, 42, 82, 57, 25, 25, 26,
       19,  8, 57, 72,  4, 49,  4, 64, 45, 19, 57, 35, 64, 19, 57, 49, 49,
       19, 57, 49,  4, 73, 64, 79, 19, 64, 75, 64, 35, 26, 19, 62, 21, 36,
       57, 25, 25, 26, 19,  8, 57, 72,  4, 49, 26, 19,  4, 45, 19, 62, 21,
       36, 57, 25, 25, 26, 19,  4, 21, 19,  4, 14, 45, 19, 52, 27, 21, 42,
       27, 57, 26, 29, 42, 42, 65, 75, 64, 35, 26, 14, 36,  4, 21])

In [81]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    #
    #print(one_hot.shape)
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [82]:
test_seq = np.array([[3, 5, 1]])

one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [83]:
# batch_size = No. of sequences per batch
def get_batches(arr,batch_size,seq_length):
    batch_size_total = batch_size * seq_length
    n_batches = len(arr) // batch_size_total
    arr = arr[:n_batches * batch_size_total]
    arr = arr.reshape((batch_size,-1))
    
    for n in range(0,arr.shape[1],seq_length):
        x = arr[:, n:n+seq_length]
        y = np.roll(x,-1)
#         y = np.zeros_like(x)
#         try:
#             y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
#         except IndexError:
#             y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

    
    

In [84]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [85]:

# printing out the first 10 items in a sequence
print('x\n', x[:22, :22])
print('\ny\n', y[:21, :21])

x
 [[50 36 57 25 14 64 35 19 34 42 42 42 82 57 25 25 26 19  8 57 72  4]
 [45 52 21 19 14 36 57 14 19 57 14 14 35 57 16 14 64 74 19 36 64 35]
 [64 21 74 19 52 35 19 57 19  8 52 64 17 19 36 64 19 57 75 52  4 74]
 [45 19 14 36 64 19 16 36  4 64  8 19 14 36 52 62 10 36 19 36  4 74]
 [19 45 57 27 19 36 64 35 19 14 64 57 35  7 45 14 57  4 21 64 74 17]
 [16 62 45 45  4 52 21 19 57 21 74 19 57 21 57 49 26 45  4 45 17 19]
 [19 24 21 21 57 19 36 57 74 19 45 57  4 74 19 14 36 57 14 19 81 52]
 [70  3 49 52 21 45 73 26 29 19 13 67 62 14 19 44 14 36 64 26 44 19]]

y
 [[36 57 25 14 64 35 19 34 42 42 42 82 57 25 25 26 19  8 57 72  4]
 [52 21 19 14 36 57 14 19 57 14 14 35 57 16 14 64 74 19 36 64 35]
 [21 74 19 52 35 19 57 19  8 52 64 17 19 36 64 19 57 75 52  4 74]
 [19 14 36 64 19 16 36  4 64  8 19 14 36 52 62 10 36 19 36  4 74]
 [45 57 27 19 36 64 35 19 14 64 57 35  7 45 14 57  4 21 64 74 17]
 [62 45 45  4 52 21 19 57 21 74 19 57 21 57 49 26 45  4 45 17 19]
 [24 21 21 57 19 36 57 74 19 45 57  4 74 19 

In [86]:
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print("Using gpu")
else:
    print("No gpu")

No gpu


In [87]:
class model(nn.Module):
    def __init__(self,tokens,n_hidden=256,n_layers=2,drop_prob=0.5,lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
    def forward(self, x, hidden):
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        return out, hidden
    
    def init_hidden(self,batch_size):
        weight = next(self.parameters()).data    
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [88]:
n_hidden=512
n_layers=2

net = model(chars, n_hidden, n_layers)
print(net)

model(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [89]:

def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [90]:
batch_size = 128
seq_length = 100
n_epochs = 2 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/2... Step: 10... Loss: 3.2385... Val Loss: 3.1787
Epoch: 1/2... Step: 20... Loss: 3.1454... Val Loss: 3.1333


KeyboardInterrupt: 

In [68]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [91]:

def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [92]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [93]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna  t   ea t ataan na  e     ea aa  an ee nte       e etneaaat   e eaa n eenae ae ttten  aennttttae  an et n ett ttene ae a n   ta a ne  nt   e  e  nt e  a n nt e nntte  a   a teeeat ta ee  t   tn te eet  eteett      n  et a    ee n   a teaaetana a  e  teeetaeta tae e aneenene a ena t ent nnee  ea eee ete aen een an aa n taa   tn neaen     eee  tne aen   e n  nn entt  t   e  t tena nte nant enena atn t tnt t aaeneeeaneae tae   aa  t an ee ee   ae an eteaaa tna a taaee a et ant ee a n naneeaat enen ta at et aa etet e   eaetn a eeenn  ee a n aeet  t ea  nn nnaettee   t nt  ean  n t at aent nnn  tnat aa an nt eatenee at aettaae aae ante  naennnn   tta  a  te etetaaee ea naa a a e ete ttaeae tea  tt a at   a e  a n teeea ne  tn et aennee attta anan n   eetane n enane netnaaaeeaant e   eaa  tn ne n  t n  eeaatata  e  nn  n neann ne   nettn t aan nae eea ta tt nt e an  e  atta en  tt    at ttae ata    a  a eee na n tt ne na a teeenanttte  t t   ee a ate   e   aa taaen n  neet aa   eteeee  

In [95]:
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f, map_location=lambda storage, loc: storage)
    
loaded = model(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [102]:
print(sample(loaded, 20000, top_k=5, prime="Voldermort"))

Voldermort, how he did not know how the clearing were
not sure how much he had ever seen in his mind, where they waited,
but his most father had still had an arrangen thing he had been. The silence flew to the
bedrooms and looked away. He could see her with the crowd on the first floor. A step with a
long, speedy soft, a flash of light and she wrestled a look of great roar of
shadow of the compartment without a strange. A few of our feet seemed to be sitting in the
concentration. Harry was sorry for him to say, before Harry would never have been
able to follow hing her to Harry and Ron, Hagrid had been carrying
trying to get through the ground with a porente comprimerth. After a
minute, Harry could hear a gleamed cold and and her shoes and cheeks
and two of the fourth-second parchment that was standing in
front of the door, as though ir hear whispering, “I think he’d be
able to say the permission between your aunt with the bashet as a cage
with a strangled contendarts of his proper sen